# Model Building for Churn Prediction

### Importing libraries

In [4]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from imblearn.combine import SMOTEENN

- Reading csv file

In [5]:
df=pd.read_csv("tel_churn.csv")
df.head()

,Unnamed: 0,SeniorCitizen,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,0,29.85,29.85,0,1,0,0,1,1,...,0,0,1,0,1,0,0,0,0,0
1,1,0,56.95,1889.50,0,0,1,1,0,1,...,0,0,0,1,0,0,1,0,0,0
2,2,0,53.85,108.15,1,0,1,1,0,1,...,0,0,0,1,1,0,0,0,0,0
3,3,0,42.30,1840.75,0,0,1,1,0,1,...,1,0,0,0,0,0,0,1,0,0
4,4,0,70.70,151.65,1,1,0,1,0,1,...,0,0,1,0,1,0,0,0,0,0


In [6]:
df=df.drop('Unnamed: 0',axis=1)

In [7]:
x=df.drop('Churn',axis=1)
x.head()

,SeniorCitizen,MonthlyCharges,TotalCharges,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,29.85,29.85,1,0,0,1,1,0,1,...,0,0,1,0,1,0,0,0,0,0
1,0,56.95,1889.50,0,1,1,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0
2,0,53.85,108.15,0,1,1,0,1,0,0,...,0,0,0,1,1,0,0,0,0,0
3,0,42.30,1840.75,0,1,1,0,1,0,1,...,1,0,0,0,0,0,0,1,0,0
4,0,70.70,151.65,1,0,1,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [8]:
y=df['Churn']
y

0       0
1       0
2       1
3       0
4       1
       ..
7027    0
7028    0
7029    0
7030    1
7031    0
Name: Churn, Length: 7032, dtype: int64

#### Train test split

In [9]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

#### Decision tree classifier

In [10]:
model_dt=DecisionTreeClassifier(criterion = "gini",random_state = 100,max_depth=6, min_samples_leaf=8)

In [11]:
model_dt.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [12]:
y_pred=model_dt.predict(x_test)
y_pred

array([0, 0, 1, ..., 0, 1, 1], dtype=int64)

In [13]:
model_dt.score(x_test,y_test)

0.7846481876332623

In [14]:
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.83      0.89      0.86      1026
           1       0.63      0.51      0.56       381

    accuracy                           0.78      1407
   macro avg       0.73      0.70      0.71      1407
weighted avg       0.77      0.78      0.78      1407



- As you can see that the accuracy is quite low, and as it's an imbalanced dataset, we shouldn't consider Accuracy as our metrics to measure the model, as Accuracy is cursed in imbalanced datasets.
Hence, we need to check recall, precision & f1 score for the minority class, and it's quite evident that the precision, recall & f1 score is too low for Class 1, i.e. churned customers.
Hence, moving ahead to call SMOTEENN (UpSampling + ENN)

In [15]:
sm = SMOTEENN()
X_resampled, y_resampled = sm.fit_sample(x,y)

In [16]:
xr_train,xr_test,yr_train,yr_test=train_test_split(X_resampled, y_resampled,test_size=0.2)

In [17]:
model_dt_smote=DecisionTreeClassifier(criterion = "gini",random_state = 100,max_depth=6, min_samples_leaf=8)

In [18]:
model_dt_smote.fit(xr_train,yr_train)
yr_predict = model_dt_smote.predict(xr_test)
model_score_r = model_dt_smote.score(xr_test, yr_test)
print(model_score_r)
print(metrics.classification_report(yr_test, yr_predict))

0.934801016088061
              precision    recall  f1-score   support

           0       0.92      0.93      0.92       511
           1       0.94      0.94      0.94       670

    accuracy                           0.93      1181
   macro avg       0.93      0.93      0.93      1181
weighted avg       0.93      0.93      0.93      1181



In [19]:
print(metrics.confusion_matrix(yr_test, yr_predict))

[[474  37]
 [ 40 630]]


We can see quite better results, i.e. Accuracy: 92 %, and a very good recall, precision & f1 score for minority class.

### Random Forest Classifier

In [20]:
from sklearn.ensemble import RandomForestClassifier

In [21]:
model_rf=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)

In [22]:
model_rf.fit(x_train,y_train)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [23]:
y_pred=model_rf.predict(x_test)

In [24]:
model_rf.score(x_test,y_test)

0.783226723525231

In [26]:
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.81      0.91      0.86      1026
           1       0.65      0.44      0.52       381

    accuracy                           0.78      1407
   macro avg       0.73      0.68      0.69      1407
weighted avg       0.77      0.78      0.77      1407



In [27]:
sm = SMOTEENN()
X_resampled1, y_resampled1 = sm.fit_sample(x,y)

In [28]:
xr_train1,xr_test1,yr_train1,yr_test1=train_test_split(X_resampled1, y_resampled1,test_size=0.2)

In [29]:
model_rf_smote=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)

In [30]:
model_rf_smote.fit(xr_train1,yr_train1)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [31]:
yr_predict1 = model_rf_smote.predict(xr_test1)

In [32]:
model_score_r1 = model_rf_smote.score(xr_test1, yr_test1)

In [33]:
print(model_score_r1)
print(metrics.classification_report(yr_test1, yr_predict1))

0.9348757497857755
              precision    recall  f1-score   support

           0       0.96      0.89      0.92       511
           1       0.92      0.97      0.94       656

    accuracy                           0.93      1167
   macro avg       0.94      0.93      0.93      1167
weighted avg       0.94      0.93      0.93      1167



In [34]:
print(metrics.confusion_matrix(yr_test1, yr_predict1))

[[456  55]
 [ 21 635]]


With RF Classifier, also we are able to get quite good results, infact better than Decision Tree.

#### Performing PCA

In [35]:
# Applying PCA
from sklearn.decomposition import PCA
pca = PCA(0.9)
xr_train_pca = pca.fit_transform(xr_train1)
xr_test_pca = pca.transform(xr_test1)
explained_variance = pca.explained_variance_ratio_

In [36]:
model=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)

In [37]:
model.fit(xr_train_pca,yr_train1)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [38]:
yr_predict_pca = model.predict(xr_test_pca)

In [39]:
model_score_r_pca = model.score(xr_test_pca, yr_test1)

In [40]:
print(model_score_r_pca)
print(metrics.classification_report(yr_test1, yr_predict_pca))

0.7292202227934875
              precision    recall  f1-score   support

           0       0.69      0.71      0.70       511
           1       0.77      0.75      0.76       656

    accuracy                           0.73      1167
   macro avg       0.73      0.73      0.73      1167
weighted avg       0.73      0.73      0.73      1167



With PCA, we couldn't see any better results, hence let's finalise the model which was created by RF Classifier

### Pickling the model

In [41]:
import pickle

In [42]:
filename = 'model.sav'

In [43]:
pickle.dump(model_rf_smote, open(filename, 'wb'))

In [44]:
load_model = pickle.load(open(filename, 'rb'))

In [45]:
model_score_r1 = load_model.score(xr_test1, yr_test1)

In [46]:
model_score_r1

0.9348757497857755

Our final model i.e. RF Classifier with SMOTEENN, is now ready and dumped in model.sav, which we will use and prepare API's so that we can access our model from UI.